# Atividade 2

```{admonition} Atividade
Ajuste do modelo de regressão linear simples

Considerando o conjunto de dados da atividade da semana passada (disponível http://leg.ufpr.br/~walmes/data/digital-cameras.txt).

1) Escolha três potenciais covariáveis e obtenha as estimativas de minimos quadrados dos coeficientes de regressão (sem usar a função lm) usando apenas a aritmética básica do R. Note que você vai estimar três modelos de regressão linear simples, ou seja, apenas um covariável em cada modelo.

2) Para cada modelo calcule os erros padrões e estatística t interprete as estimativas e os testes de hipóteses associados.
3) Confira os resultados obtidos em 1) e 2) usando a função lm do R.

4) Se vc tivesse que escolher apenas um entre os três modelos que ajustou qual você escolheria? Justifique sua resposta.
```

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as scypy_stats

from IPython.display import display, Markdown
from utils import read_cameras_dataset

cameras_dataframe = read_cameras_dataset()

Funções necessárias para definir o modelo, erros, estatísticas de teste e p-valores:

In [2]:
def get_parametros_regressao_linear(x: pd.Series, y: pd.Series) -> tuple[np.number, np.number]:
    media_x: np.number = x.mean()
    media_y: np.number = y.mean()
    diff_x_media_x: pd.Series = x - media_x

    beta_um: np.number = (diff_x_media_x * y).sum() / (diff_x_media_x ** 2).sum()
    beta_zero: np.number = media_y - beta_um * media_x
    
    return (beta_zero, beta_um)

def get_variancia_dos_parametros(x: pd.Series, y: pd.Series, beta_zero: np.number, beta_um: np.number) -> tuple[np.number, np.number]:
    n = len(y)
    media_x = x.mean()
    soma_diff_x_media_x_quadrado: pd.Series = ((x - media_x) ** 2).sum()
    y_estimado: pd.Series = beta_zero - beta_um * x
    sigma_quadrado = ((y - y_estimado) ** 2).sum() / (n - 2)

    variancia_beta_um: np.number = sigma_quadrado / soma_diff_x_media_x_quadrado
    variancia_beta_zero: np.number = sigma_quadrado * ( (1/n) + (( media_x ** 2 ) / (soma_diff_x_media_x_quadrado)) )

    return (variancia_beta_zero, variancia_beta_um)

def get_estatisticas_de_teste_t(parametro: np.number, variancia_parametro: np.number) -> np.number:
    return parametro / np.sqrt(variancia_parametro)

def get_pvalor(estatistica_de_teste_t: np.number) -> np.number:
    return 2 * scypy_stats.t.cdf(estatistica_de_teste_t)

## 1. Escolher três potenciais covariáveis

Usando correlação entre colunas numéricas, temos:

In [3]:
correlacoes_dataframe = cameras_dataframe.select_dtypes(include=[np.number]).corr()

correlacoes_dataframe['price']

id                      0.057812
price                   1.000000
popularity             -1.000000
needs_update           -0.084738
aperture_max           -0.121661
display_size            0.291625
dslr_camera             0.186358
focal_35mm_max         -0.203198
focal_35mm_min          0.222650
focal_length_max        0.045267
focal_length_min        0.630558
fps                     0.058083
iso_max                 0.209055
iso_min                 0.115017
lens_interchangeable    0.311255
optical_zoom           -0.170697
sensor_resolution       0.611263
video_resolution_x      0.320733
video_resolution_y      0.317396
weight                  0.613811
weight_body             0.646057
weight_lens             0.750130
Name: price, dtype: float64

Selecionando as 3 maiores correlações em relação ao preço, temos:

In [4]:
# Remove 1s da diagonal principal
np.fill_diagonal(correlacoes_dataframe.values, np.nan)

maiores_correlacoes = correlacoes_dataframe.nlargest(3, 'price')

""" maiores_correlacoes = pd.DataFrame(
    correlacoes_dataframe.columns[np.argsort(-correlacoes_dataframe.values, axis=1)[:, :3]], 
    columns=['1', '2', '3'],
    index=correlacoes_dataframe.index
) """

maiores_correlacoes

,id,price,popularity,needs_update,aperture_max,display_size,dslr_camera,focal_35mm_max,focal_35mm_min,focal_length_max,...,iso_max,iso_min,lens_interchangeable,optical_zoom,sensor_resolution,video_resolution_x,video_resolution_y,weight,weight_body,weight_lens
weight_lens,-0.083297,0.750130,NaN,-0.188701,0.034703,0.292348,0.328275,0.424556,0.122022,0.694221,...,0.123798,-0.197003,NaN,0.345937,0.505139,0.104763,0.094170,0.933686,0.738124,NaN
weight_body,-0.072638,0.646057,NaN,-0.244261,0.088251,0.355524,0.530048,0.137298,0.091488,0.426577,...,0.330939,-0.366212,0.030904,0.129500,0.494293,0.102874,0.092422,0.928907,NaN,0.738124
focal_length_min,0.126567,0.630558,1.0,-0.048524,0.006691,0.319863,0.425264,-0.381107,0.667781,-0.074130,...,0.239187,0.181179,0.609192,-0.357779,0.558159,0.170194,0.167264,0.593483,0.440516,0.379422


### 1.2 Equações para obter os estimadores $\beta_{0}$ e $\beta_{1}$

In [5]:
regressao = pd.DataFrame(
    {
        'covariavel': cameras_dataframe['weight_lens'],
        'variavel_resposta': cameras_dataframe['price']
    },
    dtype=np.float64)

regressao = regressao.dropna()

beta_zero, beta_um = get_parametros_regressao_linear(regressao.covariavel, regressao.variavel_resposta)

display(beta_um)
display(beta_zero)

lm = scypy_stats.linregress(regressao.values)


95.69535318947325

133.0490453715122